In [10]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import UnidentifiedImageError


# Funktsioon mudeli laadimiseks kindlast epohhist
def load_model_from_epoch(checkpoint_dir, epoch_num):
    model_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch_num:02d}.keras")
    if os.path.exists(model_path):
        print(f"Laen mudelit failist: {model_path}")
        return tf.keras.models.load_model(model_path)
    else:
        raise FileNotFoundError(f"Faili {model_path} ei leitud.")


# Funktsioon piltide laadimiseks kaustast
def load_images_from_folder(folder_path, target_size=(224, 224)):
    images = []
    image_names = []
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        try:
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)
            images.append(img_array)
            image_names.append(img_name)
        except UnidentifiedImageError:
            print(f"Viga pildi laadimisel: {img_path}")
    return np.array(images), image_names


# Laadige mudel kindlast epohhist
checkpoint_dir = "model_checkpoints"
epoch_num = 955  # epohh, millest mudel laadida
model = load_model_from_epoch(checkpoint_dir, epoch_num)

# Kuvage mudeli struktuur
model.summary()

# Määrake piltide kaust
images_folder = "uued"  # piltide kaust

# Laadige pildid kaustast
images, image_names = load_images_from_folder(images_folder)

# Normaliseerige pildid, et need oleksid vahemikus [0, 1]
images = images / 255.0

# Ennustamine piltide kohta
predictions = model.predict(images)
predicted_classes = np.argmax(predictions, axis=1)

# Kuvame tulemused
categories = ["COVID19", "NORMAL", "PNEUMONIA", "TUBERCULOSIS"]
for i, prediction in enumerate(predictions):
    predicted_class = np.argmax(prediction)  # Kõrgeima tõenäosusega klass
    second_predicted_class = np.argsort(prediction)[
        -2
    ]  # Teise kõrgeima tõenäosusega klass
    img_name = image_names[i]
    print(
        f"Pildi nimi: {img_name}, "
        f"Ennustus: {categories[predicted_class]}, Tõenäosus: {prediction[predicted_class]:.2f}, "
        f"Teine valik: {categories[second_predicted_class]}, Tõenäosus: {prediction[second_predicted_class]:.2f}"
    )

Laen mudelit failist: model_checkpoints/model_epoch_955.keras


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         8,196 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,612,302 (90.07 MB)

 Trainable params: 8,196 (32.02 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 16,394 (64.04 KB)

Viga pildi laadimisel: uued/.DS_Store
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 647ms/step
Pildi nimi: person1_virus_13.jpeg, Ennustus: PNEUMONIA, Tõenäosus: 0.75, Teine valik: NORMAL, Tõenäosus: 0.25
Pildi nimi: normal.jpeg, Ennustus: NORMAL, Tõenäosus: 0.42, Teine valik: PNEUMONIA, Tõenäosus: 0.40
Pildi nimi: Tuberculosis-34.png, Ennustus: COVID19, Tõenäosus: 0.38, Teine valik: PNEUMONIA, Tõenäosus: 0.33
Pildi nimi: COVID19(337).jpg, Ennustus: NORMAL, Tõenäosus: 0.55, Teine valik: COVID19, Tõenäosus: 0.35
Pildi nimi: Normal_posteroanterior_(PA)_chest_radiograph_(X-ray).jpg, Ennustus: COVID19, Tõenäosus: 0.66, Teine valik: PNEUMONIA, Tõenäosus: 0.23
Pildi nimi: Tuberculosis-15.png, Ennustus: PNEUMONIA, Tõenäosus: 0.48, Teine valik: TUBERCULOSIS, Tõenäosus: 0.28
Pildi nimi: COVID19(12).jpg, Ennustus: PNEUMONIA, Tõenäosus: 0.76, Teine valik: COVID19, Tõenäosus: 0.22
Pildi nimi: person9_bacteria_40.jpeg, Ennustus: PNEUMONIA, Tõenäosus: 0.91, Teine valik: NORMAL, Tõenäosus: 0.07
